# 10. Hafta Lab-1: TF-IDF
## BGM 565: Siber Güvenlik için Makine Öğrenme Yöntemleri
### İstanbul Şehir Üni. - Bilgi Güvenliği Müh.
#### Dr. Ferhat Özgür Çatak
Bu lab çalışması kapsamında dökümanlardan **tf-idf** matrisi oluşturulacaktır.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
from urllib import request
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import numpy as np

np.set_printoptions(linewidth=160)

### Örnek bir derlem olsun. 5 adet farklı dokumandan oluşan bir küme

In [2]:
corpus = ["Elma Portakal Portakal Elma",
          "Elma Muz Elma Muz",
          "Muz Elma Muz Muz Muz Elma",
          "Muz Portakal Muz Muz Portakal Muz",
          "Muz Elma Muz Muz Portakal Muz"]

### Bu derlem için sözlük oluşturup kelime frekanslarını sayalım

In [3]:
vectorizer = CountVectorizer()
vocab_freq = vectorizer.fit_transform(corpus)
print(vectorizer.vocabulary_)
print(vocab_freq)

{'elma': 0, 'portakal': 2, 'muz': 1}
  (0, 2)	2
  (0, 0)	2
  (1, 1)	2
  (1, 0)	2
  (2, 1)	4
  (2, 0)	2
  (3, 1)	4
  (3, 2)	2
  (4, 1)	4
  (4, 2)	1
  (4, 0)	1


#### Frekansların olduğu *vocab_freq* değişkeni *sparse* bir matrisdir. Bunu *dense* halke getirelim.

In [4]:
print(vocab_freq.todense())

[[2 0 2]
 [2 2 0]
 [2 4 0]
 [0 4 2]
 [1 4 1]]


## TF-IDF matrisinin oluşturulması

In [5]:
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
corpus_vec = vectorizer.transform(corpus).todense()
corpus_vec

matrix([[0.64374446, 0.        , 0.76524053],
        [0.70710678, 0.70710678, 0.        ],
        [0.4472136 , 0.89442719, 0.        ],
        [0.        , 0.85962194, 0.51093065],
        [0.23304334, 0.93217336, 0.2770264 ]])

### Derlemde yer alan kelimelerin idf değerleri

In [6]:
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'elma': 0, 'portakal': 2, 'muz': 1}
[1.18232156 1.18232156 1.40546511]


## Internet üzerinde bulunan ekonomi ve spor haberlerinin sınıflandırılması
Bu çalışmada 11 ekonomi ve 11 spor haberi Internet üzerinde alınacaktır. HTML ve javasript etiketlerinin temizlenmesinden sonra derlem oluşturulmaktadır.

In [7]:
urls = ["http://www.bloomberght.com/yorum/cuneyt-basaran/2093520-uyuyan-dev-uyandi",
                "http://www.bloomberght.com/yorum/ali-cagataygelecege-bakis/2116104-fifa-goreve",
                "http://www.bloomberght.com/yorum/ahmet-oz/2075832-kisir-donguye-mi-giriyoruz",
                "http://www.bloomberght.com/yorum/gizem-oztok-altinsac/2113416-turkiyenin-ekonomi-sinavi",
                "http://www.bloomberght.com/yorum/ceren-dilekci/2115924-megabank-muammasi",
                "http://www.bloomberght.com/ht-yazarlar/cuneyt-basaran-2071/2116920-paketin-harcama-kalemleri",
                "http://www.bloomberght.com/ht-yazarlar/gokhan-sen/2079777-merkez-bankasina-sanatci-ruhu-gerekiyor",
                "http://www.bloomberght.com/ht-yazarlar/abdurrahman-yildirim/2116917-10-yil-once-serbest-10-yil-sonra-yasak",
                "http://www.bloomberght.com/ht-yazarlar/guntay-simsek/2116923-ihracatta-rekor-ithalatta-rekor-her-taraf-rekor",
                "http://spor.mynet.com/futbol/sampiyonlar-ligi/157184-jerome-boateng-den-cuneyt-cakir-a-elestiri.html",
                "http://www.hurriyet.com.tr/ekonomi/ihracat-nisan-ayinda-13-5-milyar-dolar-oldu-40822367",
                "http://ekonomi.haber7.com/ekonomi/haber/2614392-borcu-olanlar-dikkat-18-ay-taksit-yapilacak/?detay=1",
             "http://spor.mynet.com/fenerbahce/157181-giuliano-dan-aykut-kocaman-a-ovgu.html",
             "http://spor.mynet.com/galatasaray/157176-kaderi-degistiren-kebapcidaki-kavga.html",
             "http://spor.mynet.com/fenerbahce/157175-aziz-yildirim-in-secim-kozu.html",
             "http://spor.mynet.com/fenerbahce/157174-aykut-kocaman-dan-besiktas-iddiasi.html",
             "http://spor.mynet.com/basketbol/thy-euro-league/157173-fenerbahce-de-buyuk-surpriz.html",
             "http://spor.mynet.com/futbol/sampiyonlar-ligi/157170-mac-ozeti-real-madrid-2-2-bayern-munih-golleri-izle-cuneyt-cakir-penalti-izle.html",
             "http://spor.mynet.com/ingiltere/157168-cenk-tosun-icin-carpici-yorum-hayatta-kalmayi-basardi.html",
             "http://spor.mynet.com/besiktas/157129-besiktas-ta-atiba-nin-yerine-abdullahi-shehu.html",
             "http://www.hurriyet.com.tr/sporarena/besiktastan-olayli-fenerbahce-derbisi-kararina-itiraz-40822811",
             "http://www.hurriyet.com.tr/sporarena/devler-liginde-ilk-finalist-real-madrid-40822363"]

In [8]:
corpus = []
for url in urls:
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    soup = BeautifulSoup(raw, "lxml")
    [x.extract() for x in soup.findAll(['script', 'style'])]
    raw = soup.get_text()
    corpus.append(raw)
    

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
corpus[0]

### Bu derlem için **tf-idf** matrisi 

In [ ]:
vect_tr = TfidfVectorizer()
X= vect_tr.fit_transform(corpus)


#### Oluşan matrisin boyutları

In [ ]:
X.shape

### İlk 100 kelime ve **idf** değerleri

In [ ]:
print(list(vect_tr.vocabulary_.keys())[0:100])
print(vect_tr.idf_[0:100])

### *tf-idf* matrisi

In [ ]:
X
y = np.concatenate((np.zeros(11), np.ones(11)))
y

# SVM algoritması ile sınıflandırma

In [ ]:
clf = SVC(verbose=False, kernel="poly")
clf.fit(X,y)
y_hat =clf.predict(X)
cm = confusion_matrix(y,y_hat)
cm

# Deep Neural Netwroks ile sınıflandırma

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt

model = Sequential()
model.add(Dense(500, input_dim=X.shape[1], activation='tanh', kernel_initializer='uniform' ))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='tanh', kernel_initializer='uniform'))
model.add(Dropout(0.2))
model.add(Dense(500, activation='tanh', kernel_initializer='uniform' ))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(X, y, epochs=10, batch_size=500, verbose=1,
                    validation_split=0.2)

In [ ]:
y_hat = model.predict_classes(X)
cm = confusion_matrix(y, y_hat)
print(cm)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('dogruluk', fontsize=18)
plt.xlabel('epoch', fontsize=18)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.ylabel('kayip', fontsize=18)
plt.xlabel('epoch', fontsize=18)
plt.legend(['train', 'test'], loc='upper left')

plt.show()